### Exercício 3

Dada a barra a esforço axial com secção constante $(b \times h)$ engastada e livre com uma carga distribuída dada por $q$ pede-se calcular o deslocamento em sua extremidade livre utilizando o MEF com aproximações linear e quadrática utilizando, respectivamente, 1 elemento e 5 elementos. Apresente a dedução da matriz do elemento quadrático para este caso. Na solução numérica indique claramente a numeração dos nós e dos elementos utilizados em sua discretização e apresentando a numeração dos nós e dos elementos utilizados em sua discretização e apresentado a matriz global e o campo de deslocamento aproximado comparado com a solução exata do problema.
Dados:

* $b = 0,10m$
* $E = 10^8 \cdot T (Pa)$
* $h = 0,5 \cdot T (m)$
* $q(x) = 10 \cdot T (kN/m)$
* $l = 10 \cdot T (m)$

Com $T = \frac{20 - N}{19}$

#### Resolução

##### Substituindo $N$

Inicialmente, substituiremos os valores de $N$ e $T$:

$$
N = 0 \Rightarrow T = \frac{20}{19}
$$

Logo, temos que:

* $b = 0,10m$
* $E = 10^8 \cdot \frac{20}{19} (Pa)$
* $h = 0,5 \cdot \frac{20}{19} (m)$
* $q(x) = 10 \cdot \frac{20}{19} (kN/m)$
* $l = 10 \cdot \frac{20}{19} (m)$

##### Equação do problema

$$
-EA \frac{\partial ^2 u(x)}{\partial x^2} = q(x)
$$

Onde $0 < x < L$, $A = bh$ é a área transversal e $u$ é o vetor deslocamento.

##### Condições de Contorno

Do enunciado, temos a barra tem 

In [ ]:
# 

import sympy as sp

sp.var()